In [50]:
from torchdrug import core
from torchdrug import datasets, transforms,layers
from torchdrug.core import Registry as R
from torchdrug.layers import geometry

import torch
import torchdrug
from torchdrug import data

import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.use('Agg')

# 数据集展示

In [3]:
EnzymeCommission = R.search("datasets.EnzymeCommission")
PV = R.search("transforms.ProteinView")
trans = PV(view = "residue")
dataset = EnzymeCommission("~/scratch/protein-datasets/", test_cutoff=0.95, 
                           atom_feature="full", bond_feature="full", verbose=1, transform = trans)

23:49:36   Extracting /home/cu/scratch/protein-datasets/EnzymeCommission.zip to /home/cu/scratch/protein-datasets


Loading /home/cu/scratch/protein-datasets/EnzymeCommission/enzyme_commission.pkl.gz: 100%|██████████| 18716/18716 [00:24<00:00, 757.80it/s]


展示第一个样本的前两个残基的原子

In [4]:
# 数据集第一个样本，前两个残基的原子
protein = dataset[0]["graph"]
print(protein)
is_first_two = (protein.residue_number == 1) | (protein.residue_number == 2) | (protein.residue_number == 3)
first_two = protein.residue_mask(is_first_two, compact=True)

first_two.visualize()
#plt.savefig("fig/first_two_three.png")


Protein(num_atom=1596, num_bond=2920, num_residue=349)


/home/cu/anaconda3/envs/gearnet/lib/python3.8/site-packages/torchdrug/data/molecule.py:586: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  fig.show()


In [5]:
first_two_elements = dataset[:2]
graphs = [element["graph"] for element in first_two_elements]
protein2 = data.Protein.pack(graphs)
print(protein2)

PackedProtein(batch_size=2, num_atoms=[1596, 3761], num_bonds=[2920, 6468], num_residues=[349, 997])


测试edge_feature

In [7]:
graph = dataset[0]["graph"]
print(graph)

edge_list = graph.edge_list
print(edge_list)

num_relations = 7

node_in, node_out, _ = edge_list.t()
print("node_in: ", node_in)
print("node_out: ", node_out)
print("\n")


print("atom2residue:", graph.atom2residue)
print("atom2residue.shape:", graph.atom2residue.shape)
print("\n")

residue_in, residue_out = graph.atom2residue[node_in], graph.atom2residue[node_out]
print("residue_in: ", residue_in)
print("residue_out: ", residue_out)
print("residue_in.shape: ", residue_in.shape)
print("\n")




Protein(num_atom=1596, num_bond=2920, num_residue=349)
tensor([[   1,    0,    0],
        [   0,    1,    0],
        [   2,    1,    0],
        ...,
        [1429, 1430,    0],
        [1431, 1421,    0],
        [1421, 1431,    0]])
node_in:  tensor([   1,    0,    2,  ..., 1429, 1431, 1421])
node_out:  tensor([   0,    1,    1,  ..., 1430, 1421, 1431])


atom2residue: tensor([  0,   0,   0,  ..., 346, 347, 348])
atom2residue.shape: torch.Size([1596])


residue_in:  tensor([  0,   0,   0,  ..., 184, 184, 184])
residue_out:  tensor([  0,   0,   0,  ..., 184, 184, 184])
residue_in.shape:  torch.Size([2920])




只保留alpha碳，以及按照gearnet格式简化图

In [6]:
graph_construction_model = layers.GraphConstruction(node_layers=[geometry.AlphaCarbonNode()], 
                                                    edge_layers=[geometry.SpatialEdge(radius=10.0, min_distance=5),
                                                                 geometry.KNNEdge(k=10, min_distance=5),
                                                                 geometry.SequentialEdge(max_distance=2)
                                                                 ],
                                                    edge_feature="gearnet"
                                                    )

_protein = data.Protein.pack([protein])
protein_ = graph_construction_model(_protein)
print("Graph before: ", _protein)
print("Graph after: ", protein_)

print("node_feature: ", protein_.node_feature.shape)
print("edge_feature: ", protein_.edge_feature.shape)

print("edge_weight: ", protein_.edge_weight.shape)

print("node_position: ", protein_.node_position.shape)

# 测试unsqueeze
edge_weight = protein_.edge_weight.unsqueeze(-1)
print("new_edge_weight: ", edge_weight.shape, edge_weight)



Graph before:  PackedProtein(batch_size=1, num_atoms=[1596], num_bonds=[2920], num_residues=[349])
Graph after:  PackedProtein(batch_size=1, num_atoms=[185], num_bonds=[3754], num_residues=[185])
node_feature:  torch.Size([185, 21])
edge_feature:  torch.Size([3754, 59])
edge_weight:  torch.Size([3754])
node_position:  torch.Size([185, 3])
new_edge_weight:  torch.Size([3754, 1]) tensor([[1.],
        [1.],
        [1.],
        ...,
        [1.],
        [1.],
        [1.]])


# Gearnet 流程

生成稀疏的邻接矩阵

In [7]:
node_in, node_out, relation = protein_.edge_list.t()
node_out = node_out * protein_.num_relation + relation
print(node_in, node_out)
print(node_in.shape, node_out.shape)

import torchdrug.utils as utils
adjacency = utils.sparse_coo_tensor(torch.stack([node_in, node_out]), protein_.edge_weight,
                                    (protein_.num_node, protein_.num_node * protein_.num_relation))

print(adjacency)
print(adjacency.shape)

tensor([90, 85, 83,  ..., 75, 50, 51]) tensor([575, 547, 547,  ..., 749, 756, 756])
torch.Size([3754]) torch.Size([3754])
tensor(indices=tensor([[ 90,  85,  83,  ...,  75,  50,  51],
                       [575, 547, 547,  ..., 749, 756, 756]]),
       values=tensor([1., 1., 1.,  ..., 1., 1., 1.]),
       size=(185, 1295), nnz=3754, layout=torch.sparse_coo)
torch.Size([185, 1295])


# DGM模块

In [66]:
protein2_ = graph_construction_model(protein2)
graph = protein2_
#graph = protein_
print("Graph before: ", protein2)
print("Graph after: ", graph)
print("\n")
print("graph_node_feature :",graph.node_feature.shape)
print("graph_edge_feature :",graph.edge_feature.shape)
print("\n")
print("graph_node _feature_type :",graph.node_feature.dtype)

len(graph.node_feature)
#print(graph.node_feature.max())

Graph before:  PackedProtein(batch_size=2, num_atoms=[1596, 3761], num_bonds=[2920, 6468], num_residues=[349, 997])
Graph after:  PackedProtein(batch_size=2, num_atoms=[185, 415], num_bonds=[3754, 8999], num_residues=[185, 415])


graph_node_feature : torch.Size([600, 21])
graph_edge_feature : torch.Size([12753, 59])


graph_node _feature_type : torch.int64


600

In [84]:
dataloader = data.DataLoader(graph, batch_size=2, shuffle=True)

In [85]:
from torchdrug.layers import functional

# 定义一个简单的模型，包含SelfAttentionBlock
class SimpleModel(nn.Module):
    def __init__(self, input_dim, num_heads, hidden_dim):
        super(SimpleModel, self).__init__()
        self.attention = layers.SelfAttentionBlock(hidden_dim = 512, num_heads = 8,dropout=0.1)
        self. linear = layers.MultiLayerPerceptron(input_dim, 
                                                    hidden_dims = [512, 512], 
                                                    short_cut=True, 
                                                    batch_norm=True, 
                                                    activation="relu")
        self.hidden_dim = hidden_dim

    
    def forward(self, graph):  
         
        input = graph.node_feature.to(torch.float32)  # (num_nodes, input_dim)
        mlp_output = self.linear(input)  # (num_nodes, hidden_dim)
    
        attn_input, sizes = functional._extend(mlp_output, graph.num_residues, mlp_output, graph.num_residues)
        print("attn_input: ", attn_input.shape)
        print("sizes: ", sizes)
        attn_input, mask = functional.variadic_to_padded(attn_input, sizes)
        print("mask: ", mask.shape)
        print("attn_input: ", attn_input.shape)
        
        output = self.attention(mlp_output, mask)
        #output = self.attention(mlp_output, mask)
        return output


In [86]:
# 定义模型
input_dim = graph.node_feature.shape[-1]
num_heads = 8
hidden_dim = 512
model = SimpleModel(input_dim, num_heads, hidden_dim)

# 获取一个批次的数据并传递给模型
for batch in dataloader:
    #graph = batch['graph']
    output = model(graph)
    print("Output:", output)
    print("Output shape:", output.shape)
    break  # 这里只查看一个批次

attn_input:  torch.Size([1200, 512])
sizes:  tensor([370, 830])
mask:  torch.Size([2, 830])
attn_input:  torch.Size([2, 830, 512])


ValueError: not enough values to unpack (expected 3, got 2)

# Gearnet测试

In [56]:
from torchdrug import models

dataloader = data.DataLoader(graph, batch_size=1, shuffle=True)

In [58]:
GN = models.GearNet(input_dim=21,
                    hidden_dims=512,
                    batch_norm=True,
                    concat_hidden=True,
                    short_cut=True,
                    readout="sum",
                    num_relation=7,
                    edge_input_dim=59
                    )

# 获取一个批次的数据并传递给模型
for batch in dataloader:
    #graph = batch['graph']
    gearnet_output= GN(graph, graph.node_feature.to(torch.float32))
    node_feature1 = gearnet_output["node_feature"]
    #print("hiddden_node_feature:", node_feature1.shape)

    break  # 这里只查看一个批次